# Demonstrating Different Decoding Strategies with Hugging Face Transformers
In this notebook, we will explore various decoding strategies for text generation using a small encoder-decoder model from Hugging Face's Transformers library. We'll apply these strategies to two different datasets:

- Translation task where deterministic strategies are expected to perform better.
- Summarization task where stochastic strategies might yield more diverse and informative outputs.

The decoding strategies we'll test include:

1. Greedy Search
2. Beam Search
3. Temperature Sampling
4. Top-k Sampling
5. Top-p (Nucleus) Sampling

We'll define a custom ```generate_text``` function to apply these strategies and evaluate their performance using appropriate metrics for each dataset.

Here are some useful links you might want to check:
- [Auto Classes](https://huggingface.co/docs/transformers/model_doc/auto)
- [transformers\AutoModel](https://huggingface.co/docs/transformers/model_doc/auto#transformers.AutoModel)
- [transformers\AutoTokenizer](https://huggingface.co/docs/transformers/model_doc/auto#transformers.AutoTokenizer)
- [Google's T5](https://arxiv.org/pdf/1910.10683)
- [T5-small](https://huggingface.co/google-t5/t5-small)
- [Flan-T5-small](https://huggingface.co/google/flan-t5-small)

In [4]:
!pip install transformers datasets sacrebleu rouge_score evaluate torch --quiet

In [9]:
import sys
!"{sys.executable}" -m pip install torch transformers datasets sacrebleu rouge_score evaluate --quiet

c:\Users\ulise\Desktop\Comillas\4ºiMAT\Procesamiento Lenguaje Natural 2\Prácticas\P3\p3-transformer-Ulisesdz\.venv\Scripts\python.exe



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset
import random
import evaluate

c:\Users\ulise\Desktop\Comillas\4ºiMAT\Procesamiento Lenguaje Natural 2\Prácticas\P3\p3-transformer-Ulisesdz\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load the Pre-trained Model and Tokenizer

We'll use the ```flan-T5-small``` model, which is a small encoder-decoder model suitable for both story_gen and summarization tasks. This model is based on Google's ```t5-small``` model, but fine-tuned on more than 1000 additional tasks covering also more languages.

In [18]:
model_name = 'google/flan-t5-small'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [19]:
def generate_text(input_text, strategy='greedy', max_length=50, **kwargs):
    """Generates text based on the specified decoding strategy.

    Args:
        input_text (str): The input text to be processed by the model.
        strategy (str, optional): The decoding strategy to use. Defaults to 'greedy'.
            Options include:
            - 'greedy': Greedy search decoding.
            - 'beam': Beam search decoding.
            - 'temperature': Temperature sampling.
            - 'top-k': Top-k sampling.
            - 'top-p': Top-p (nucleus) sampling.
            - 'contrastive': Contrastive search decoding.
        max_length (int, optional): The maximum length of the generated text. Defaults to 50.
        **kwargs: Additional keyword arguments specific to the decoding strategy.

    Keyword Args:
        num_beams (int, optional): Number of beams for beam search. Defaults to 5.
            Used when `strategy='beam'`.
        temperature (float, optional): Sampling temperature. Defaults to 1.0.
            Used when `strategy='temperature'`.
        top_k (int, optional): The number of highest probability vocabulary tokens to keep for top-k filtering. Defaults to 50.
            Used when `strategy='top-k'` or when `strategy='contrastive'`.
        top_p (float, optional): Cumulative probability for nucleus sampling. Defaults to 0.95.
            Used when `strategy='top-p'`.
        penalty_alpha (float, optional): Contrastive search penalty factor. Defaults to 0.6.
            Used when `strategy='contrastive'`.

    Returns:
        str: The generated text based on the decoding strategy.

    Raises:
        ValueError: If an unknown decoding strategy is specified.
    """
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    model.eval()
    with torch.no_grad():
        if strategy == 'greedy':
            output_ids = model.generate(input_ids, max_length=max_length)
        elif strategy == 'beam':
            num_beams = kwargs.get('num_beams', 5)
            output_ids = model.generate(
                input_ids, max_length=max_length, num_beams=num_beams, early_stopping=True
            )
        elif strategy == 'temperature':
            temperature = kwargs.get('temperature', 1.0)
            output_ids = model.generate(
                input_ids, max_length=max_length, do_sample=True, temperature=temperature
            )
        elif strategy == 'top-k':
            top_k = kwargs.get('top_k', 50)
            output_ids = model.generate(
                input_ids, max_length=max_length, do_sample=True, top_k=top_k
            )
        elif strategy == 'top-p':
            top_p = kwargs.get('top_p', 0.95)
            output_ids = model.generate(
                input_ids, max_length=max_length, do_sample=True, top_p=top_p
            )
        elif strategy == 'contrastive':
            penalty_alpha = kwargs.get('penalty_alpha', 0.6)
            top_k = kwargs.get('top_k', 4)
            output_ids = model.generate(
                input_ids, max_length=max_length, penalty_alpha=penalty_alpha, top_k=top_k
            )
        else:
            raise ValueError("Unknown strategy: {}".format(strategy))

    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output_text


### Use case 1: Neural Machine Translation
The WMT16 English-German dataset is a collection of parallel sentences in English and German used for machine translation tasks. It is part of the Conference on Machine story_gen (WMT) shared tasks, which are benchmarks for evaluating machine story_gen systems. The dataset contains professionally translated sentences and covers a variety of topics, making it ideal for training and evaluating story_gen models.

We'll use a subset (1% of the test split) of the dataset for evaluation purposes.

In [14]:
dataset_nmt = load_dataset('wmt16', 'de-en', split='test[:1%]')

c:\Users\ulise\Desktop\Comillas\4ºiMAT\Procesamiento Lenguaje Natural 2\Prácticas\P3\p3-transformer-Ulisesdz\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ulise\.cache\huggingface\hub\datasets--wmt16. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the '

In [21]:
# Randomly select a sample from the dataset
sample = random.choice(dataset_nmt['translation'])
input_text = sample['de']
target_text = sample['en']

print("Input Text:")
print(input_text)
print("\nTarget Text:")
print(target_text)


Input Text:
Die beiden wollten über die Umsetzung der internationalen Vereinbarung sowie über Teherans destabilisierende Maßnahmen im Nahen Osten sprechen.

Target Text:
The two wanted to talk about the implementation of the international agreement and about Teheran's destabilising activities in the Middle East.


We need to preprocess the dataset to prepare it for input into the T5 model. The T5 model expects input in a specific format, including a task prefix. This is because the T5 model is a multi-purpose model, being able to perform several task, so we need to tell it what it must do.

In [22]:
def preprocess_translation(examples: list[dict]) -> dict:
    """Preprocesses a single example for the translation task.

    Args:
        examples list[dict]: A list of dictionaries containing 'de' and 'en' keys with English and German sentences.

    Returns:
        dict[list]: A dictionary of lists with added 'src_texts' and 'tgt_texts' keys for model input and target.

    The function:
    - Adds the task prefix 'translate German to English: ' to the German sentence.
    - Stores the result in 'src_texts'.
    - Copies the English sentence to 'tgt_texts'.
    """
    # Empty dictionary
    texts = {}
    
    # T5 expects a "translate German to English: " prefix
    texts['src_texts'] = ['translate German to English: ' + ex['de'] for ex in examples]
    texts['tgt_texts'] = [ex['en'] for ex in examples]
    return texts

dataset_nmt_preproc = preprocess_translation(dataset_nmt["translation"])


Let's see how we can generate a translation using the t5 model. From a random dataset, we are going to create translations using the different implemented strategies:

In [24]:
# Obtain source and target texts
random_index = random.randint(0, len(dataset_nmt_preproc['src_texts']))
random_src_sentence = dataset_nmt_preproc['src_texts'][random_index]
random_tgt_sentence = dataset_nmt_preproc['tgt_texts'][random_index]

# Obtain the translated sentence
translation_greedy= generate_text(random_src_sentence, strategy='greedy')
translation_beam_search = generate_text(random_src_sentence, strategy='beam', num_beams=5)
translation_temperature = generate_text(random_src_sentence, strategy='temperature', temperature=0.7)
translation_top_k = generate_text(random_src_sentence, strategy='top-k', top_k=50)
translation_top_p = generate_text(random_src_sentence, strategy='top-p', top_p=0.95)
# translation_contrastive = generate_text(random_src_sentence, strategy='contrastive', top_k=4, penalty_alpha=0.6)

print(random_src_sentence)
print(f"Original translation: \n{random_tgt_sentence}")
print(f"Greedy Search: \n{translation_greedy}")
print(f"Beam Search: \n{translation_beam_search}")
print(f"Temperature Sampling: \n{translation_temperature}")
print(f"Top-k Sampling: \n{translation_top_k}")
print(f"Top-p Sampling: \n{translation_top_p}")
# print(f"Contrastive Search: \n{translation_contrastive}")

translate German to English: Es gab keinen Hinweis darauf, dass Lamb, der in zwei Online-Kursen für die Delta State University in Cleveland, Mississippi, unterrichtete, bereits 300 Meilen zu dem Schulgelände gereist war, wo er nach Angaben der Polizei einen beliebten Geschichtsprofessor, Ethan Schmidt, an der Tür zu seinem Büro erschossen und getötet hat.
Original translation: 
There was no indication that Lamb, who was teaching two online classes for Delta State University in Cleveland, Mississippi, had already traveled 300 miles to the school's campus, where police believe he shot and killed a well-liked history professor, Ethan Schmidt, in the doorway to his office.
Greedy Search: 
It is not clear whether Lamb, which was launched online for Delta State University in Cleveland, Mississippi, has been rehabilitated by police, who have uncovered a prominent geologist, Ethan Schmidt, who has been buried and
Beam Search: 
It is not clear whether the Lamb, which is in two online courses fo

- Do you see something different between the deterministic and the stochastic strategies? Try different random sentences.

En los resultados se nota claramente que las estrategias deterministas (Greedy y Beam Search) generan siempre la misma traducción, con frases más correctas pero algo rígidas o repetitivas. Tienden a mantener la coherencia, aunque a veces pierden naturalidad o fluidez en el texto.


En cambio, las estrategias estocásticas (Temperature, Top-k y Top-p) introducen aleatoriedad, por lo que cada ejecución produce resultados distintos. Sus traducciones son más variadas y suenan más naturales, aunque a veces menos precisas o con pequeños errores de significado.

### Evaluation Metric: BLEU Score
#### What is BLEU Score?
The BLEU (Bilingual Evaluation Understudy) score is a metric for evaluating the quality of text that has been machine-translated from one language to another. It compares a candidate translation to one or more reference translations and calculates a score based on the overlap of n-grams (contiguous sequences of words).

BLEU-4: Considers up to 4-gram matches between the candidate and reference translations. It provides a balance between precision (matching words) and fluency (maintaining the structure of the language). Using BLEU-4 allows us to capture not just individual word matches (unigrams) but also phrases of up to four words. This makes the evaluation more sensitive to the quality of the translation in terms of both accuracy and fluency.

We'll use the ```sacrebleu``` implementation for a standardized BLEU score calculation (which is BLEU-4). You can check the details [here](https://aclanthology.org/W14-3346.pdf).

In [25]:
bleu_metric = evaluate.load('sacrebleu')

In [27]:
strategies = ['greedy', 'beam', 'temperature', 'top-k', 'top-p']    #, 'contrastive'
results = {}

hyperparameters = {
    'num_beams': 5,
    'temperature': 1.0,
    'top_k': 50,
    'top_p': 0.95,
    'penalty_alpha': 0.6
}

for strategy in strategies:
    print(f"Evaluating strategy: {strategy}")
    predictions = []
    references = []
    for src_text, tgt_text in zip(dataset_nmt_preproc["src_texts"], dataset_nmt_preproc["tgt_texts"]): 
        pred = generate_text(src_text, strategy=strategy, **hyperparameters)
        predictions.append(pred)
        references.append([tgt_text])  # SacreBLEU expects a list of references

    # Compute BLEU-4 score
    bleu = bleu_metric.compute(predictions=predictions, references=references, smooth_method='exp')
    results[strategy] = bleu['score']
    print(f"BLEU-4 score for {strategy}: {bleu['score']:.2f}")


Evaluating strategy: greedy
BLEU-4 score for greedy: 20.99
Evaluating strategy: beam
BLEU-4 score for beam: 21.14
Evaluating strategy: temperature
BLEU-4 score for temperature: 10.42
Evaluating strategy: top-k
BLEU-4 score for top-k: 8.11
Evaluating strategy: top-p
BLEU-4 score for top-p: 10.44


- Seeing the above translations and the BLEU score of the different strategies, which strategy would you choose for this use case?

Viendo los resultados, las estrategias deterministas (Greedy y Beam Search) obtienen los BLEU más altos y, por tanto, las traducciones más cercanas a las referencias del conjunto de datos. En cambio, las estrategias estocásticas (Temperature, Top-k y Top-p) logran puntuaciones mucho más bajas, lo que indica mayor variabilidad y menor fidelidad a las traducciones originales.


Por esto mismo, este caso donde se busca precisión y consistencia en la traducción la mejor opción sería Beam Search, ya que ofrece la puntuación BLEU más alta.

### Use case 2: Story generation

The ```WritingPrompts``` dataset is a collection of imaginative prompts and corresponding stories from the Reddit community. It contains over 300,000 stories written in response to various prompts, making it suitable for training and evaluating models on creative text generation tasks.

We'll use a subset of the dataset for evaluation purposes.

In [28]:
dataset_st_gen = load_dataset('llm-aes/writing-prompts', split='train[:1%]')

c:\Users\ulise\Desktop\Comillas\4ºiMAT\Procesamiento Lenguaje Natural 2\Prácticas\P3\p3-transformer-Ulisesdz\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ulise\.cache\huggingface\hub\datasets--llm-aes--writing-prompts. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for t

In [29]:
# Randomly select a sample from the dataset
sample = random.choice(dataset_st_gen['prompt'])

print("Sample Text:")
print(sample)

Sample Text:
 You live a , `` normal , '' life , and one day you drift into sleep , your brain suddenly creates a dream verse . ( Expansion in comments )



Now, we have to tell the t5 model to generate text after the input sentence.

In [30]:
def preprocess_story_generation(examples: list[dict]) -> dict:
    """Preprocesses a single example for the story generation task.

    Args:
        examples (list[dict]): A list of dictionaries containing 'prompt' key.

    Returns:
        dict[list]: A dictionary of list with added 'src_texts' for model input.

    The function:
    - Adds the task prefix 'Write a story based on: ' to the prompt.
    """
    # Empty dictionary
    texts = {}
    
    # T5 expects a task prefix
    texts['src_texts'] = ['Write a story based on: ' + ex['prompt'] for ex in examples]
    return texts

dataset_st_gen_preproc = preprocess_story_generation(dataset_st_gen)


As we have no original story to compare how good the model generates a story, you should compare the different decoding strategies by looking at some random stories: 

In [32]:
# Obtain source and target texts
random_index = random.randint(0, len(dataset_st_gen_preproc['src_texts']))
random_src_sentence = dataset_st_gen_preproc['src_texts'][random_index]

# Obtain the translated sentence
story_gen_greedy= generate_text(random_src_sentence, max_length=300, strategy='greedy')
story_gen_beam_search = generate_text(random_src_sentence, max_length=300, strategy='beam', num_beams=5)
story_gen_temperature = generate_text(random_src_sentence, max_length=300, strategy='temperature', temperature=0.7)
story_gen_top_k = generate_text(random_src_sentence, max_length=300, strategy='top-k', top_k=50)
story_gen_top_p = generate_text(random_src_sentence, max_length=300, strategy='top-p', top_p=0.95)
# story_gen_contrastive = generate_text(random_src_sentence, max_length=300, strategy='contrastive', top_k=4, penalty_alpha=0.6)

print(random_src_sentence)
print(f"Greedy Search: \n{story_gen_greedy}")
print(f"Beam Search: \n{story_gen_beam_search}")
print(f"Temperature Sampling: \n{story_gen_temperature}")
print(f"Top-k Sampling: \n{story_gen_top_k}")
print(f"Top-p Sampling: \n{story_gen_top_p}")
# print(f"Contrastive Search: \n{story_gen_contrastive}")

Write a story based on:  Upon his suicide Hitler will join the dead waiting to be ferried across the river Styx . The dead include the millions who died in WWII . You have been tasked with keeping Hitler safe until he can arrive for his final judgment .

Greedy Search: 
Hitler was a savior . He was a savior . He was a savior . He was a savior . He was a savior . He was a savior . He was a savior . He was a savior . He was a savior . He was a savior . He was a savior . He was a savior . He was a savior . He was a savior . He was a savior . He was a savior . He was a savior . He was a savior . He was a savior . He was a savior . He was a savior . He was a savior . He was a savior . He was a savior . He was a savior . He was a savior . He was a savior . He was a savior . He was a savior . He was a savior 
Beam Search: 
Upon his suicide , Hitler will join the dead waiting to be ferried across the river Styx . The dead include the millions who died in WWII . You have been tasked with keepin

- Seeing the above generated stories of the different strategies, which strategy would you choose for this use case?

En este caso, la estrategia Beam Search sería la más adecuada. Su salida es coherente, mantiene el sentido del texto original y no presenta repeticiones absurdas ni contenido fuera de contexto, como ocurre con otras estrategias.


Las demás muestran problemas claros: Greedy Search repite frases de manera continua, Temperature, Top-k y Top-p generan respuestas inventadas. En cambio, Beam Search ofrece un texto más estable y fiel al prompt.

### Sensitivity Analysis

Try different hyperparameter values for the decoding strategies, try to optimize the BLEU score for the Neural Machine Translation case and generate better stories in the second use case.

- Which optimal configuration have you found for use case 1? Which are your conclusions based on your analysis?

Después de probar distintos valores de hiperparámetros, la mejor configuración fue Beam Search con num_beams = 5 y temperature = 1.0. Esta combinación dio el BLEU-4 más alto y dio traducciones más precisas y naturales. Aumentar el número de beams no mejoró el rendimiento y solo hizo el proceso más lento, mientras que las estrategias basadas en sampling (como top-k o top-p) generaban resultados menos consistentes. En resumen, Beam Search es la estrategia más equilibrada para este tipo de tareas de traducción.

- Which optimal configuration have you found for use case 2? Which are your conclusions bsaed on your analysis?

En el caso de la generación de texto, los mejores resultados se obtuvieron con Top-p Sampling, usando top_p = 0.9 y temperature = 0.8. Esta configuración generó historias más coherentes y variadas, evitando la monotonía del Greedy Search y la aleatoriedad excesiva del Top-k, ya que introducir cierta aleatoriedad en este contexto mejora la creatividad, que es lo que buscamos.